# Evaluate trained model

## Load model

In [ ]:
import os
!ls -l {os.environ.get('RESULTS_DIR')}

In [ ]:
run = "colab01"
SAVE_DIR = f"{os.environ.get('RESULTS_DIR')}/{run}"

In [ ]:
from ddsp.colab import jupyter_utils
import ddsp.training
import gin
from matplotlib import pyplot as plt
import numpy as np

# Parse the gin config.
gin_file = os.path.join(SAVE_DIR, 'operative_config-0.gin')
gin.parse_config_file(gin_file)

data_provider = ddsp.training.data.TFRecordProvider(os.environ.get("URMP_MONO"))
dataset = data_provider.get_batch(batch_size=1, shuffle=False)

try:
  batch = next(iter(dataset))
except OutOfRangeError:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')


# Load model
model = ddsp.training.models.Autoencoder()
model.restore(SAVE_DIR)

jupyter_utils.show_audio(batch['audio'][0])
# Resynthesize audio.
outputs = model(batch, training=False)
audio_gen = model.get_audio_from_outputs(outputs)
audio = batch['audio'][0]

print('Original Audio')
jupyter_utils.show_audio(batch['audio'][0])

print('Resynthesis')
jupyter_utils.show_audio(outputs['audio_synth'][0])

# Timbre interpolation